Student's Name and Email Address

Boise State University, Department of Chemistry and Biochemistry

## CHEM 324: PChem Lab {-}
# Worksheet 7: Fluorescence Quantum Yield {-}

In [ ]:
# Import the main modules used in this worksheet
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression

In [ ]:
# I am assuming the file in question will be located in a `Quantumyield_Data/` subfolder in your `Colab Notebook/` folder. 
# Make sure to change these names if you use different files.
# Load the google drive with your files 
from google.colab import drive
drive.mount('/content/drive')
# The following needs to be the path of the folder with all your datafile in .csv format
path='/content/drive/MyDrive/Colab Notebooks/QuantumYield_Data/'

## Task 1: Emission and Excitation Spectra {-}

* Visualize the chemical structures of quninine sulfate (CAS 6119-70-6) and fluorescein (CAS 2321-07-5) and comment on why these molecules are fluorophores/dyes.
* Plot the emission and excitation spectra for fluorescein and quinine. You will need to present one figure for each molecule, with two spectra per figure. 
* Report the integrated areas for each spectrum of each molecule.
* Explicitly report all the other constants and/or known experimental values that you will need to compute the quantum yield at infinite dilution of fluorescein. 

## Task 2: Quantum Yield of the Standard {-}

The Stern-Volmer equation for self-quenching of solutions allows you to compute the quantum yield of fluorescence from the molar concentration of the dye, $C$, as  

$\frac{\phi _0}{\phi _f}=1+14.5\cdot C$

where $\phi _0$ is the quantum yield at infinite dilution.

* Given that for quinine you know $\phi _0=0.545$, use the Stern-Volmer equation to compute the quantum yield of quinine, $\phi _f$, at the two concentrations used in the experiments. 

## Task 3: Quantum yield of the Unknown {-}

Although the absolute incident intensity of the radiation is unkonwn, the relative incident intensity between an unknown sample and a standard sample will be the same. The quantum yield of the analyte, fluorescein, can thus be determined by comparing the intesity of fluorescence with the one of quinine. 

$\frac{\phi ^u_f}{\phi ^s_f}=\frac{I^u_f\cdot(1-10^{-A_s})}{I^u_s\cdot(1-10^{-A_u})}$

where $I^{u/s}_f$ is the area under the emission spectrum of the unknown/standard (i.e. fluorescein/quinine), while $A_{s/u}$ is the absorbance if the standard/unknown measured at the excitation wavelength. 

* Use the formula above and the results from the previous tasks to compute the fluorescence quantum yield of fluorescein at the two concentrations used in the experiments 

## Task 4: Stern-Volmer Curve for Fluorescein {-}

* Perform a linear fit of the fluorescence quantum yield of fluorescein as a function of concentration. From the intercept of your linear fit you should obtain the fluorescence quantum yield at infinite dilution for fluorescein.
* Plot the Stern-Volmer curve for fluorescein.